In [1]:
import os
os.environ['JAX_PLATFORMS'] = 'cpu'

In [2]:
from gemma import transformer, sampler
import dataclasses

# import orbax.checkpoint as ocp
# checkpointer = ocp.StandardCheckpointer()
# params = checkpointer.restore("/nfs/nfs2/users/dibya/gemma-2/gemma2-2b/")

# model = transformer.Gemma.create_from_pretrained(params, dataclasses.replace(transformer.GemmaConfig.gemma2_2b(), dtype="bfloat16"))

In [3]:
def test_sampling():
    model = transformer.Gemma.create(jax.random.PRNGKey(0), dataclasses.replace(transformer.GemmaConfig.gemma2_2b(), dtype="bfloat16"))
    sampler.sample(model, jnp.ones((1, 1024), dtype=jnp.int32), jnp.ones((1, 1024), dtype=jnp.int32), 1024, 1, jax.random.PRNGKey(0))

In [4]:
import jax
import jax.numpy as jnp
jax.eval_shape(test_sampling)

1 1024
(Traced<ShapedArray(bfloat16[26,1,2048,4,256])>with<DynamicJaxprTrace>, Traced<ShapedArray(bfloat16[26,1,2048,4,256])>with<DynamicJaxprTrace>)


TypeError: while_loop body function carry input and carry output must have equal types (e.g. shapes and dtypes of arrays), but they differ:

  * the input carry component state.last_positions has type int32[1] but the corresponding output carry component has type int32[1,1024], so the shapes do not match;
  * the input carry component state.done has type bool[1,1] but the corresponding output carry component has type bool[1,1024], so the shapes do not match.

Revise the function so that all output types (e.g. shapes and dtypes) match the corresponding input types.

In [4]:
import flax
import jax
import jax.numpy as jnp
params = flax.traverse_util.flatten_dict(params, sep="/")


In [5]:

new_params = {k: v for k, v in params.items() if "transformer/layer_" not in k}

for k in filter(lambda x: "transformer/layer_0/" in x, params.keys()):
    new_params[k.replace("/layer_0/", "/layers/")] = jnp.stack([
        params[k.replace("layer_0", f"layer_{i}")]
        for i in range(26)
    ])
new_params = flax.traverse_util.unflatten_dict(new_params, sep="/")

In [6]:
import jax
jax.tree.map(lambda x: x.shape, new_params)

{'transformer': {'embedder': {'input_embedding': (256128, 2304)},
  'final_norm': {'scale': (2304,)},
  'layers': {'attn': {'attn_vec_einsum': {'w': (26, 8, 256, 2304)},
    'kv_einsum': {'w': (26, 2, 4, 2304, 256)},
    'q_einsum': {'w': (26, 8, 2304, 256)}},
   'mlp': {'gating_einsum': {'w': (26, 2, 2304, 9216)},
    'linear': {'w': (26, 9216, 2304)}},
   'post_attention_norm': {'scale': (26, 2304)},
   'post_ffw_norm': {'scale': (26, 2304)},
   'pre_attention_norm': {'scale': (26, 2304)},
   'pre_ffw_norm': {'scale': (26, 2304)}}}}

In [7]:
import nnjax
flattened_target = flax.traverse_util.flatten_dict(nnjax.asdict(model))
flattened_src = flax.traverse_util.flatten_dict(new_params)

In [8]:
print("keys in target but not in src:")
print(set(flattened_target.keys()) - set(flattened_src.keys()))
print("keys in src but not in target:")
print(set(flattened_src.keys()) - set(flattened_target.keys()))


keys in target but not in src:
{('embedder', 'input_embedding')}
keys in src but not in target:
{('transformer', 'embedder', 'input_embedding')}
